# Image Classification Pipeline using Remo

![remo_logo](assets/remo_normal.png)

In this tutorial, Remo will be used to accelerate the process of building a transfer learning pipeline for the task of Image Classification.

In [ ]:
import sys
%load_ext autoreload
%autoreload 2
# Specify path to Remo
# Mac version
local_path_to_repo =  '/home/harsha/Documents/remo-python'
# Windows version
#local_path_to_repo =  'C:/Users/crows/Documents/GitHub/remo-python'

sys.path.insert(0, local_path_to_repo)

In [45]:
#Imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.models as models
import torch.optim as optim
import numpy as np
from PIL import Image
import pandas as pd
import os
import tqdm
from pprint import pprint
import json
import glob
import random
random.seed(4)

import remo
remo.set_viewer('jupyter')



## Adding Data to Remo

- The dataset used in this example is a subset of the <a href="http://www.robots.ox.ac.uk/~vgg/data/flowers/102/">Flowers 102 Dataset</a>.
- The next cell will download the data as a zip file and extract the files in a new folder.

- The folder structure of the dataset is:

    ```
    ├── small_flowers
        ├── images
            ├── 1
                ├── image_1.jpg
                ├── image_2.jpg
                ├── ...
            ├── 2
                ├── image_3.jpg
                ├── image_4.jpg
                ├── ...
            ├── ... 
        ├── annotations
        ├── mapping.json
    ```

In [46]:
# The dataset will be extracted in a new folder
if not os.path.exists('small_flowers.zip'):
    !wget https://s-3.s3-eu-west-1.amazonaws.com/small_flowers.zip
    !unzip small_flowers.zip
else:
    print('Files already downloaded')

Files already downloaded


In [48]:
# The path to the folders
path_to_images =  './small_flowers/images/'
mapping_json_path =  './small_flowers/mapping.json'

### Annotations

We can easily generate annotations from a series of folders, by passing the root directory path to ```remo.generate_annotations_from_folders(). ``` 

In [ ]:
remo.generate_annotations_from_folders(path_to_images)

To visualise the labels as strings rather than IDs, we can use a dictionary mapping the two of them.
In the donwnloaded dataset, we havev JSON file containing this information

In [50]:
cat_to_index =  dict(json.load(open(mapping_json_path)))

### Train / test split

To organise our images, we can also generate a list of annotation tags. 
Among other things, this allows to generate train / test splits without the need to move around image files. 
To do this, we just need to pass a dictionary that maps tags to the relevant images paths to the function ```remo.generate_tags_from_folders()```.

In [ ]:
im_list = [os.path.basename(i) for i in glob.glob(path_to_images+'/**/*.jpg', recursive=True)]
im_list = random.sample(im_list, len(im_list))

# Definining the train test split
train_idx = round(len(im_list) * 0.8)
valid_idx = train_idx + round(len(im_list) * 0.1)
test_idx  = valid_idx + round(len(im_list) * 0.1)

# Creating a dictionary with tags
tags_dict =  {'train' : im_list[0:train_idx], 
              'valid' : im_list[train_idx:valid_idx], 
              'test' : im_list[valid_idx:test_idx]}

remo.generate_image_tags(tags_dictionary  = tags_dict, 
                         output_file_path = os.path.join(path_to_images, 'images_tags.csv'))

### Creating a dataset

To add a dataset, you can use the ```remo.create_dataset()``` specifying the path to data and annotations.

The class encoding is passed via a dictionary.

For a complete list of formats supported please refer the <a href="https://remo.ai/docs/annotation-formats/">documentation</a>.


In [ ]:
# The annotations.csv is generated in the same path of the sub-folder
flowers =  remo.create_dataset(name = 'flowers', 
                              local_files = [path_to_images],
                              annotation_task = 'Image classification',
                              class_encoding = cat_to_index)

**Visualizing the dataset**

To view your data and labels using the Remo visual interface directly in the notebook, call the ```dataset.view()``` method.

In [ ]:
flowers.view()

![dataset_view](assets/flower_dataset_view.png)

**Dataset Statistics**

Remo alleviates the need to write extra boilerplate for accessing dataset properties. 

This can be done either using code, or via the visual interface.

In [54]:
flowers.get_annotation_statistics()

[{'AnnotationSet ID': 226,
  'AnnotationSet name': 'Image classification',
  'n_images': 140,
  'n_classes': 3,
  'n_objects': 0,
  'top_3_classes': [{'name': 'Hard-leaved pocket orchid', 'count': 60},
   {'name': 'Canterbury bells', 'count': 40},
   {'name': 'Pink primrose', 'count': 40}],
  'creation_date': None,
  'last_modified_date': '2020-08-19T06:13:08.258898Z'}]

In [ ]:
flowers.view_annotation_stats()

![view_annotations_stats](assets/flower_dataset_view_annotation_stats.png)

**Export Annotations**

Using the ```dataset.export_annotations_to_file()``` method, annotations can be exported to a format of your choice.

For a complete list of formats supported please refer the <a href="https://remo.ai/docs/annotation-formats/">documentation</a>.


In [ ]:
flowers.export_annotations_to_file('flowers_annotations.zip', 
                                   annotation_format = 'csv', 
                                   full_path = True, 
                                   export_tags = True)

annotation_for_model_path = './small_flowers/annotations/for_model/'
   
if not os.path.exists(annotation_for_model_path):
    os.makedirs(annotation_for_model_path)
    
!unzip flowers_annotations.zip -d './small_flowers/annotations/for_model/'

## Feeding Data into PyTorch

A custom PyTorch ```Dataset``` object defined below is used to load data.

In order to adapt this to your dataset, the following are required:
- **Path to Tags:** Path to tags csv file for Train, Test, Validation split generated by Remo
- **Path to Annotations:** Path to Annotations csv file (Format : file_name, class_name)
- **Mapping:** Python dictionary containing mapping of class name and class index (Format : {'class_name' : 'class_index"})
- **transforms:** Transforms to be applied to the images before passing it to the network.

In [ ]:
# Mapping from Class Name --> Index
mapping = { value : key for (key, value) in cat_to_index.items()}

In [ ]:
class FlowerDataset(Dataset):
    
    def __init__(self, annotations, train_test_valid_split, mapping, mode, transform=None):
        # Pandas is used to read in the csv file into a DataFrame for data loading
        self.data = pd.read_csv(annotations)
        self.train_test_valid_split = pd.read_csv(train_test_valid_split)
        self.data['tag'] = self.train_test_valid_split['tag']
        self.mapping = mapping
        self.transform = transform
        self.mode = mode
        
        i_index = self.data['tag'] == self.mode
        self.data_df = self.data[i_index][['file_name', 'classes']].reset_index(drop=True)
        
    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        labels = int(self.mapping[self.data_df.loc[idx, 'classes'].lower()])
        im_path = self.data_df.loc[idx, 'file_name']
        
        label_tensor =  torch.as_tensor(labels-1, dtype=torch.long)
        im = Image.open(im_path)
    
        if self.transform:
            im = self.transform(im)
    
        if self.mode == 'test':
            # For saving the predicitions, the file name is required
            return {'im' : im, 'labels': label_tensor, 'im_name' : self.data_df.loc[idx, 'file_name']}
        else:
            return {'im' : im, 'labels' : label_tensor}

In [ ]:
# Channel wise mean and standard deviation for normalizing according to ImageNet Statistics
means =  [0.485, 0.456, 0.406]
stds  =  [0.229, 0.224, 0.225]

# Transforms to be applied to Train-Test-Validation
train_transforms =  transforms.Compose([
                    transforms.RandomRotation(30),
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.ToTensor(),
                    transforms.Normalize(means, stds)])

test_valid_transforms =  transforms.Compose([
                         transforms.Resize(224),
                         transforms.CenterCrop(224),
                         transforms.ToTensor(),
                         transforms.Normalize(means, stds)])

The train, test and validation datasets are instantiated and wrapped around a ```DataLoader``` method.

In [ ]:
train_dataset = FlowerDataset(annotations =  annotation_for_model_path + 'Image classification.csv',
                              train_test_valid_split =  annotation_for_model_path + 'tags.csv',
                              transform =  train_transforms,
                              mapping =  mapping,
                              mode =  'train')

valid_dataset = FlowerDataset(annotations = annotation_for_model_path + 'Image classification.csv',
                              train_test_valid_split = annotation_for_model_path + 'tags.csv',
                              transform = test_valid_transforms,
                              mapping = mapping,
                              mode = 'valid')

test_dataset  = FlowerDataset(annotations = annotation_for_model_path + 'Image classification.csv',
                              train_test_valid_split = annotation_for_model_path + 'tags.csv',
                              transform = test_valid_transforms,
                              mapping = mapping,
                              mode = 'test')

# If you face issues in operating systems like Windows, you can set num_workers=0.
train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   =  torch.utils.data.DataLoader(valid_dataset, batch_size=1,  shuffle=False, num_workers=2)
test_loader  =  torch.utils.data.DataLoader(test_dataset,batch_size=1, shuffle=False, num_workers=2)

## Training the Model

We use a ```ResNet-18``` architecture, with weight pre-trained on ImageNet.

To train the model, the following details are to be specified:

1. **Model:** The edited version of the pre-trained model.
2. **Data Loaders:** The dictionary containing our training and validation dataloaders
3. **Criterion:** The loss function used for training the network
4. **Num_epochs:** The number of epochs for which we would like to train the network.
5. **dataset_size:** an additional parameter which is used to correctly scale the loss, the method for this is specified in the DataLoader cell

In [ ]:
model = models.resnet18(pretrained=True)

# Freezing the weights
for param in model.parameters():
    param.required_grad = False


# Replacing the final layer
model.fc =  nn.Sequential(nn.Linear(512, 256), 
                         nn.ReLU(), 
                         nn.Dropout(p=0.5), 
                         nn.Linear(256, 102), 
                         nn.LogSoftmax(dim=1))

In [ ]:
# Model Parameters

optimizer    =  optim.Adam(model.fc.parameters(), lr=0.001)
criterion    =  nn.NLLLoss()
num_epochs   =  1
data_loaders =  {'train' : train_loader, 'valid': val_loader}
dataset_size =  {'train' : len(train_dataset), 'valid' : len(valid_dataset)}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device) # This method pushes the model to the device.


In [ ]:
# The training loop trains the model for the total number of epochs.
# An epoch is one complete pass over the entire dataset

for epoch in range(num_epochs):
    
    model.train() # This sets the model back to training after the validation step
    print('\nEpoch Number {}''.format(epoch+1))

    training_loss = 0.0
    val_loss = 0.0
    val_acc = 0
    correct_preds = 0
    best_acc = 0
    validation = 0.0
    total = 0
    
    data_loader = tqdm.tqdm(data_loaders['train'])
    
    for x, data in enumerate(data_loader):
        inputs, labels = data['im'].to(device), data['labels'].to(device)
        outputs = model(inputs)
        optimizer.zero_grad()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    
    epoch_loss = training_loss / dataset_size["train"]
    print('Training Loss : {:.5f}'.format(epoch_loss))
    val_data_loader = tqdm.tqdm(data_loaders['valid'])
    
    # Validation step after every epoch
    # The gradients are not required at inference time, hence the model is set to eval mode
    with torch.no_grad():
        model.eval()
        for x, data in enumerate(val_data_loader):
            inputs, labels = data["im"].to(device), data["labels"].to(device)
            outputs = model(inputs)

            val_loss = criterion(outputs, labels)
            _, index = torch.max(outputs, 1)

            total += labels.size(0)
            correct_preds += (index == labels).sum().item()

            validation += val_loss.item()

        val_acc = 100 * (correct_preds / total)

        print('Validation Loss : {:.5f}'.format(validation / dataset_size['valid']))
        print('Validation Accuracy is: {:.2f}%'format(val_acc))


            

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.eval()
tk0 =  tqdm.tqdm(test_loader)

total =  0
correct_preds =  0
pred_list =  {}

with torch.no_grad():
    for x, data in enumerate(tk0):
        single_im, label = data['im'].to(device), data['labels'].to(device)
        im_name = data['im_name']
        
        pred = model(single_im)

        _, index = torch.max(pred, 1)

        total += label.size(0)
        correct_preds += (index == label).sum().item()
        
        pred_list[os.path.basename(im_name[0])] = cat_to_index[str((index+1).item())]
        
df = pd.DataFrame(pred_list.items(), columns=['file_name', 'class_name'])
with open('./small_flowers/results.csv', 'w') as f:
    df.to_csv(f, index=False)
print('Accuracy of the network on the test images: %d %%' % (100 * (correct_preds / total)))

## Visualizing Predictions

Using Remo, we can visualize predictions  vs the original labels.

To do this we create a new ```AnnotationSet```, and  upload predictions as a csv file

In [ ]:
flowers.create_annotation_set(annotation_task = 'Image Classification', 
                              name = 'model_predictions', 
                              path_to_annotation_file = './small_flowers/results.csv')

In [ ]:
flowers.view()

![Results Comparison](assets/flower_dataset_view_predictions.png)